In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import MySQLdb
from MySQLdb.constants import FIELD_TYPE

my_conv = { FIELD_TYPE.DECIMAL: float }
db=MySQLdb.connect(host="localhost",db="ercot_data",read_default_file="/etc/mysql/my.cnf",conv=my_conv)
c=db.cursor()

In [35]:
c.execute("""SELECT DISTINCT settlement_point FROM DAM_prices_by_SPP""")
r = c.fetchall()
r = list(r)
settlement_points = []
for settlement_point in r:
    if settlement_point[0] == "\"Settlement Point\"":
        continue
    settlement_points.append(settlement_point[0])

In [36]:
result_matrix = []
for (idx,val) in enumerate(settlement_points):
    c.execute("""SELECT delivery_date,hour_ending,spp 
                FROM DAM_prices_by_SPP 
                WHERE settlement_point = "%s" 
                AND delivery_date > "2011-01-01" 
                AND delivery_date < "2011-12-31" 
                ORDER BY delivery_date,hour_ending""" % val)
    result = c.fetchall()
    result = list(result)
    result_matrix.append(result)

In [37]:
result_matrix
spp_matrix = []
dt_matrix = []
for sp in result_matrix:
    spps = []
    dts = []
    for (date,time,spp) in sp:
        time = str(int(time.split(":")[0])-1)
        dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
        spps.append(float(spp))
        dts.append(dt)
    spp_matrix.append(spps)
    dt_matrix.append(dts)

In [38]:
for (idx,val) in enumerate(spp_matrix):
    plt.plot(dt_matrix[0],val,label="%s" % settlement_points[idx])
plt.title("SPP by LZ and HUB for 2011")
plt.xlabel("Date-Time")
plt.ylabel("SPP")
plt.show()